# Function Calling in OpenAI - Maersk Edition
## Learn Custom Tools and Function Calling

### Why Function Calling?

Models alone can't solve all problems. We need to give them tools:
- **Web search tool** - Get latest information from the web
- **Calculator tool** - Do mathematical calculations
- **Calendar tool** - Manage schedules
- **Custom business tools** - Access internal systems

### How It Works:

1. Define tool schema using JSON Schema
2. Embed tool schema in the prompt
3. Model decides when to call functions
4. Execute functions and return results
5. Model generates final response

**Reference:** [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)

---

## Setup: Install Dependencies & Configure API Key

**Choose ONE of the following methods:**

### Method 1: Colab Secrets (Recommended)
1. Click the key icon (🔑) in the left sidebar
2. Add a secret named: `OPENAI_API_KEY`
3. Paste your API key value

### Method 2: Hardcoded (Quick Start)
Directly set your API key in the code below (not recommended for production)

In [ ]:
# Install required packages
!pip install openai -q

In [ ]:
from openai import OpenAI
import json
import os

# =====================================================
# METHOD 1: HARDCODED API KEY (Uncomment to use)
# =====================================================
# OPENAI_API_KEY = "sk-proj-your-api-key-here"  # Replace with your actual API key
# client = OpenAI(api_key=OPENAI_API_KEY)
# print("✅ API Key loaded from hardcoded value")

# =====================================================
# METHOD 2: COLAB SECRETS (Recommended)
# =====================================================
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ API Key loaded from Colab Secrets")
except:
    # Fallback: Try environment variable
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    if not OPENAI_API_KEY:
        print("⚠️ WARNING: No API key found!")
        print("Please uncomment METHOD 1 above and add your API key, or use Colab Secrets")
        raise ValueError("OpenAI API key not configured")
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ API Key loaded from environment variable")

print("\n🎯 OpenAI Client Ready for Function Calling!")

---
# Lesson 1: Custom Tools and Function Calling

## Step 1: Define Tool Schema

We'll create a shipment tracking tool for Maersk containers.

In [ ]:
# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_shipment_status",
        "description": "Get the current status and location of a shipment by container number.",
        "parameters": {
            "type": "object",
            "properties": {
                "container_number": {
                    "type": "string",
                    "description": "The container tracking number (e.g., MAEU1234567)",
                },
            },
            "required": ["container_number"],
        },
    },
]

print("✅ Tool schema defined")
print("\n📋 Tool Details:")
print(json.dumps(tools[0], indent=2))

## Step 2: Define the Actual Python Function

This is the function that will be called when the model decides to use the tool.

In [ ]:
def get_shipment_status(container_number):
    """
    Simulates fetching shipment status from a database.
    In production, this would query your actual tracking system.
    """
    # Mock database response
    return f"{container_number}: Your container is currently at Mumbai Port, scheduled to depart on 15th October."

# Test the function
test_result = get_shipment_status("MAEU1234567")
print("🧪 Function Test:")
print(test_result)

## Step 3: First API Call - Model Decides to Use Tool

We send the user's question to the model along with the tool definition.

In [ ]:
# Create the conversation history
input_list = [
    {"role": "user", "content": "Where is my shipment? Container number is MAEU1234567."}
]

print("👤 User Question:")
print(input_list[0]["content"])
print("\n⏳ Sending to model with tool definitions...\n")

# 2. Prompt the model with tools defined
resp1 = client.responses.create(
    model="gpt-5-nano",
    tools=tools,
    input=input_list,
    instructions="Use Tools if you find it necessary."
)

print("🤖 Model Response (First Call):")
print(f"Response type: {type(resp1)}")
print(f"Number of outputs: {len(resp1.output)}")

## Step 4: Examine the Model's Tool Call

Let's see what the model decided to do.

In [ ]:
# Examine the response
print("🔍 Examining Model's Decision:\n")

for item in resp1.output:
    print(f"Output type: {item.type}")
    
    if item.type == "function_call":
        print(f"\n✅ Model decided to call a function!")
        print(f"  Function name: {item.name}")
        print(f"  Call ID: {item.call_id}")
        print(f"  Arguments: {item.arguments}")
        
        # Parse the arguments
        args = json.loads(item.arguments)
        print(f"\n📦 Extracted container number: {args['container_number']}")

## Step 5: Append Model's Tool Call to Conversation

We need to add the model's tool call decision to our conversation history.

In [ ]:
# Append model's structured tool call
for item in resp1.output:
    if item.type == "function_call":
        input_list.append({
            "type": "function_call",
            "call_id": item.call_id,
            "name": item.name,
            "arguments": item.arguments
        })

print("✅ Tool call appended to conversation history")
print(f"\n📚 Conversation history now has {len(input_list)} items")

## Step 6: Execute the Function and Append Result

Now we actually call our Python function and add the result to the conversation.

In [ ]:
# Execute the tool and append its output
for item in resp1.output:
    if item.type == "function_call" and item.name == "get_shipment_status":
        print("⚙️ Executing function...\n")
        
        # Parse JSON arguments to dictionary
        args = json.loads(item.arguments)
        
        # Extract the container number
        container_number = args["container_number"]
        print(f"📞 Calling: get_shipment_status('{container_number}')")
        
        # Run the actual Python function
        result = get_shipment_status(container_number)
        print(f"\n📤 Function returned: {result}")

        # Append function result to conversation
        input_list.append({
            "type": "function_call_output",
            "call_id": item.call_id,  # Must match the call_id from the function call
            "output": result  # Plain string is fine
        })

print(f"\n✅ Function output appended to conversation")
print(f"📚 Conversation history now has {len(input_list)} items")

## Step 7: Second API Call - Model Generates Final Answer

Now we send everything back to the model so it can generate a user-friendly response.

In [ ]:
print("⏳ Sending function result back to model...\n")

# Second API call - model gives final answer
resp2 = client.responses.create(
    model="gpt-5",
    instructions="Respond only with the shipment status returned by the tool. Do not add or modify anything.",
    tool_choice="none",  # Ensures no further tool calling
    input=input_list,
)

print("✅ Final Response Generated!")
print("="*60)

## Step 8: Display Final Result

In [ ]:
print("\n" + "="*60)
print("FINAL OUTPUT TO USER")
print("="*60)
print(resp2.output_text)
print("="*60)

## Optional: View Full Response JSON

For debugging, you can examine the complete response structure.

In [ ]:
print("🔍 Full Response JSON (for debugging):")
print(resp2.model_dump_json(indent=2))

---
## Understanding the Flow

Let's visualize what happened in our conversation:

In [ ]:
print("\n📊 Complete Conversation Flow:\n")
print("="*60)

for idx, item in enumerate(input_list, 1):
    print(f"\nStep {idx}:")
    
    if isinstance(item, dict):
        if "role" in item:
            print(f"  Role: {item['role']}")
            print(f"  Content: {item['content'][:100]}..." if len(item.get('content', '')) > 100 else f"  Content: {item.get('content')}")
        elif "type" in item:
            print(f"  Type: {item['type']}")
            if item['type'] == 'function_call':
                print(f"  Function: {item['name']}")
                print(f"  Arguments: {item['arguments']}")
            elif item['type'] == 'function_call_output':
                print(f"  Output: {item['output'][:100]}..." if len(item['output']) > 100 else f"  Output: {item['output']}")

print("\n" + "="*60)

---
## Exercise: Create Your Own Function

Try creating a new function for checking port schedules!

In [ ]:
# TODO: Define a new tool for checking port schedules
port_schedule_tools = [
    {
        "type": "function",
        "name": "get_port_schedule",
        "description": "Get the departure and arrival schedule for a specific port.",
        "parameters": {
            "type": "object",
            "properties": {
                "port_name": {
                    "type": "string",
                    "description": "The name of the port (e.g., Mumbai, Rotterdam)",
                },
            },
            "required": ["port_name"],
        },
    },
]

# TODO: Implement the function
def get_port_schedule(port_name):
    # Add your implementation here
    schedules = {
        "Mumbai": "Next departure: 18th October at 14:00 hrs. Next arrival: 16th October at 09:30 hrs.",
        "Rotterdam": "Next departure: 20th October at 08:00 hrs. Next arrival: 19th October at 16:00 hrs.",
    }
    return schedules.get(port_name, f"No schedule information available for {port_name}")

# TODO: Test your function
print("🧪 Testing port schedule function:")
print(get_port_schedule("Mumbai"))

---
## Summary

### What You Learned:

1. ✅ **Tool Schema Definition** - How to define function schemas using JSON
2. ✅ **Function Implementation** - Creating Python functions that can be called
3. ✅ **Two-Step Process**:
   - First API call: Model decides to use tool
   - Execute function and get result
   - Second API call: Model generates user-friendly response
4. ✅ **Conversation Management** - Maintaining conversation history with tool calls

### Key Concepts:

- **tools**: Array of function definitions following OpenAI's schema
- **function_call**: Model's decision to call a function
- **function_call_output**: The result returned by your function
- **tool_choice**: Control whether model can call tools (`"auto"`, `"none"`, or specific function)

### Next Steps:

- Try adding multiple tools
- Implement real database queries
- Add error handling for function calls
- Combine function calling with RAG (Retrieval Augmented Generation)

### Resources:

- [OpenAI Function Calling Guide](https://platform.openai.com/docs/guides/function-calling)
- [OpenAI API Reference](https://platform.openai.com/docs/api-reference)
- [JSON Schema Documentation](https://json-schema.org/)

---

**Happy Coding! 🚢**